In [8]:
import json
from typing import Dict, List

def restructure_moodle_course(course_data: List[Dict]) -> List[Dict]:
    """
    Chuyển đổi JSON từ API core_course_get_contents thành cấu trúc phân cấp.
    Các subsection sẽ được gộp vào section cha tương ứng, loại bỏ module subsection khỏi modules.
    
    Args:
        course_data: Danh sách các section từ JSON của Moodle.
    
    Returns:
        Danh sách các section đã được tổ chức phân cấp.
    """
    # Tạo từ điển để ánh xạ section id và itemid với section
    section_map = {section['id']: section for section in course_data}
    subsection_to_parent = {}
    
    # Xác định các subsection và section cha
    for section in course_data:
        if section.get('component') == 'mod_subsection' and section.get('itemid'):
            subsection_to_parent[section['itemid']] = section['id']
            print(f"Found subsection: {section['name']} (ID: {section['id']}, itemid: {section['itemid']})")
    
    # Tạo cấu trúc mới
    new_structure = []
    
    for section in course_data:
        # Bỏ qua các subsection (sẽ được xử lý trong section cha)
        if section.get('component') == 'mod_subsection':
            print(f"Skipping subsection: {section['name']} (ID: {section['id']})")
            continue
            
        # Sao chép thông tin section, loại bỏ key 'subsections' nếu đã tồn tại
        new_section = section.copy()
        new_section.pop('subsections', None)  # Loại bỏ subsections từ JSON gốc
        new_section['subsections'] = []
        
        # Tạo danh sách module mới, loại bỏ module subsection
        new_modules = [module for module in section.get('modules', []) if module.get('modname') != 'subsection']
        
        # Tìm tất cả các module có customdata chứa sectionid liên kết tới section này
        for module in section.get('modules', []):
            if module.get('modname') == 'subsection':
                try:
                    customdata = json.loads(module.get('customdata', '{}'))
                    subsection_id = customdata.get('sectionid')
                    # Chuyển đổi subsection_id thành số nguyên
                    try:
                        subsection_id = int(subsection_id) if subsection_id else None
                    except (ValueError, TypeError):
                        subsection_id = None
                    print(f"Processing module: {module['name']} (ID: {module['id']}, sectionid: {subsection_id})")
                    if subsection_id and subsection_id in section_map:
                        # Thêm subsection vào section cha
                        subsection = section_map[subsection_id].copy()
                        subsection['parent_section_id'] = section['id']
                        # Loại bỏ key 'subsections' nếu tồn tại trong subsection
                        subsection.pop('subsections', None)
                        new_section['subsections'].append(subsection)
                        print(f"Added subsection: {subsection['name']} (ID: {subsection_id}) to section {section['name']}")
                    else:
                        print(f"Warning: No valid subsection_id found for module {module['name']} (ID: {module['id']})")
                except json.JSONDecodeError:
                    print(f"Warning: Invalid customdata in module {module['id']}: {module.get('customdata')}")
        
        new_section['modules'] = new_modules
        new_structure.append(new_section)
    
    return new_structure

# Hàm để in cấu trúc phân cấp ra màn hình (dùng để kiểm tra)
def print_hierarchical_structure(structure: List[Dict], indent: int = 0):
    """
    In cấu trúc phân cấp ra màn hình để dễ đọc.
    
    Args:
        structure: Danh sách các section đã được tổ chức.
        indent: Mức thụt đầu dòng.
    """
    for section in structure:
        print('  ' * indent + f"Section: {section['name']} (ID: {section['id']})")
        # In các module trực tiếp trong section
        if section.get('modules'):
            for module in section['modules']:
                print('  ' * (indent + 1) + f"Module: {module['name']} (Type: {module['modname']})")
        # In các subsection và module bên trong
        if section.get('subsections'):
            print('  ' * (indent + 1) + "Subsections:")
            for subsection in section['subsections']:
                print('  ' * (indent + 2) + f"Subsection: {subsection['name']} (ID: {subsection['id']})")
                for module in subsection.get('modules', []):
                    print('  ' * (indent + 3) + f"Module: {module['name']} (Type: {module['modname']})")

# Hàm để đọc dữ liệu từ file JSON
def load_json_file(file_path: str) -> List[Dict]:
    """
    Đọc dữ liệu JSON từ file.
    
    Args:
        file_path: Đường dẫn tới file JSON.
    
    Returns:
        Dữ liệu JSON dưới dạng danh sách các dictionary.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if not isinstance(data, list):
            print(f"Error: Data in '{file_path}' is not a list of sections.")
            return []
        print(f"Loaded {len(data)} sections from {file_path}")
        return data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except json.JSONDecodeError:
        print(f"Error: File '{file_path}' is not a valid JSON.")
        return []

# Ví dụ sử dụng
if __name__ == "__main__":
    # Đường dẫn tới file JSON
    file_path = 'course.json'  # Cập nhật tên file theo tên bạn đã sử dụng
    
    # Đọc dữ liệu từ file JSON
    course_data = load_json_file(file_path)
    
    if course_data:
        # Chuyển đổi cấu trúc
        new_structure = restructure_moodle_course(course_data)
        
        # In kết quả để kiểm tra
        print("\nHierarchical Structure:")
        print_hierarchical_structure(new_structure)
        
        # Lưu cấu trúc mới vào file JSON
        output_file = 'restructured_course.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(new_structure, f, ensure_ascii=False, indent=2)
        print(f"\nCấu trúc mới đã được lưu vào '{output_file}'")
        
        # In thông tin kiểm tra
        print(f"\nTotal sections processed: {len(new_structure)}")
        for section in new_structure:
            print(f"Section {section['id']} has {len(section['subsections'])} subsections and {len(section.get('modules', []))} direct modules")

Loaded 10 sections from course.json
Found subsection: Bài 2.1: Lớp và đối tượng (ID: 17, itemid: 5)
Found subsection: Bài 2.2: Tính kế thừa và đa hình (ID: 18, itemid: 6)
Found subsection: Bài 1.1: Cấu trúc chương trình và biến (ID: 19, itemid: 7)
Found subsection: Bài 1.2:  Câu lệnh điều kiện và vòng lặp (ID: 20, itemid: 8)
Processing module: Bài 1.1: Cấu trúc chương trình và biến (ID: 15, sectionid: 19)
Added subsection: Bài 1.1: Cấu trúc chương trình và biến (ID: 19) to section Chương 1: Lập trình Java căn bản
Processing module: Bài 1.2:  Câu lệnh điều kiện và vòng lặp (ID: 16, sectionid: 20)
Added subsection: Bài 1.2:  Câu lệnh điều kiện và vòng lặp (ID: 20) to section Chương 1: Lập trình Java căn bản
Processing module: Bài 2.1: Lớp và đối tượng (ID: 12, sectionid: 17)
Added subsection: Bài 2.1: Lớp và đối tượng (ID: 17) to section Chương 2: Lập trình Hướng đối tượng với Java 
Processing module: Bài 2.2: Tính kế thừa và đa hình (ID: 13, sectionid: 18)
Added subsection: Bài 2.2: Tín